# COMPATIBILITY EXAMPLE

In this notebook we provide an example to show how the `LocalBasis` class is compatible with both the `weighted_POD` and `DOD` classes.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from dlroms import*
from dlroms.roms import*
import dlroms.fespaces as fe

import numpy as np
from numpy import pi
import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(1, '../code')

from PODlib import *
from variabilitylib import *

## Data loading

In [ ]:
# DATA
dataset = np.load("../data/nstokes_data.npz")
mu, u = dv.tensor(dataset['mu']), dv.tensor(dataset['u'])

ndata, ntimes, nh = u.shape
p = mu.shape[-1]

# MESH
mesh = fe.loadmesh("../data/nstokes_mesh.xml")
Vh = fe.space(mesh, 'CG', 2)

# REPARAMETRIZATION
epsilon = np.power(10, mu[:, 0])
rho = mu[:, 1]
eta = epsilon / rho

mu[:, 0] = (1/rho)
mu[:, 1] = eta

# RESHAPE TO MAKE TIME A PARAMETER
mut = dv.zeros(ndata, ntimes, p+1)
times = dv.tensor(np.linspace(0, 3.5, ntimes))
for i in range(ndata):
    mut[i,:,:2] = mu[i]
    mut[i,:, 2] = times

u = u.reshape(-1, nh)
mut = mut.reshape(-1, p+1)

# NUMBER OF TRAINING DATA
ntrain = 31*ntimes

# SET SEED FOR MONTE CARLO ESTIMATES
seed = 42

## Ambient space setup

In [ ]:
nA = 100
V, svalues = POD(u[:ntrain], k = nA)
A = gramschmidt(V.unsqueeze(0)).squeeze(0)

uA = projectdown(A, u).squeeze(-1)

## DOD class definition

In [ ]:
class DOD(DFNN):
    def __init__(self, root, branch, n = 1, A = None, trainable = True, qr = QRgramschmidt):
        super(DOD, self).__init__(root, branch**n, n = n, A = A, trainable = trainable, qr = qr)
        
    def forward(self, x):
        out = self.qr(super(DOD, self).forward(x).transpose(1,2)).transpose(1,2)
        return out if self.trainable else out.matmul(self.A)
    
    def innerdod(self, x):
        return self.qr(super(DOD, self).forward(x).transpose(1,2)).transpose(1,2)
    
    def freeze(self):
        super(DOD, self).freeze()
        self.trainable = False
        
    def basis(self, mu):
        if(self.trainable):
            raise RuntimeError("Cannot produce a hierarchically sorted basis during training. Please freeze the model first.")
        return POD(self.solve(mu), self.n, inner = self.inner)[0]

## DOD neural network definition

We consider the case where $\mu = (1/\rho,t)$.

In [ ]:
from dlroms.dnns import Reshape
from dlroms.dnns import Fourier

class Separated(ROM):
    def forward(self, x):
        return ((self[0](self.geom(x)))*(self[1](self.phys(x)))).sum(axis = -1)

In [ ]:
pbad = 2
pgood = (p+1) - pbad
mu_bad = mut[:, [0,2]]

# We define the architectures for the seed and the roots neural network
arch1 = Dense(1, 50) + Dense(50, 8*12) + Reshape(8, 12)
arch2 = Fourier(4) + Dense(9, 30) + Dense(30, 8*12) + Reshape(8, 12)
psi = Separated(arch1, arch2, geom = lambda m: m[:, [0]], phys = lambda m: m[:, [1]])

root = Dense(8, 50) + Dense(50, 50) + Dense(50, nA, activation = None)

dod = DOD(psi, root, n = 4, A = A)
dod.He()

# Error and loss
def DODerror(utrue, Vpred):
  upred = project(Vpred, utrue, orth = False)
  return (euclidean(utrue-upred).reshape(-1, ntimes).sum(axis = -1)/
          euclidean(utrue).reshape(-1, ntimes).sum(axis = -1)        ).mean()

def DODloss(utrue, Vpred):
  upred = project(Vpred, utrue, orth = False)
  return (euclidean(utrue-upred, squared = True).reshape(-1, ntimes).sum(axis = -1)).mean()

# Optimizer
optimizer = torch.optim.Adam

Once the DOD object has been defined, we can load an already trained neural network.

In [ ]:
dod.load("../results/Test_DOD.npz")

## `weighted_POD`

In [ ]:
n = 4 # we take the same number of basis the DOD has been trained for
lambda_penalty = 5e-1

w_POD = weighted_POD(A=A,
                     U=torch.t(uA[:ntrain,:]),
                     theta_full=torch.t(mut[:ntrain,:]),
                     n_basis=n,
                     omega_func=lambda theta, theta_i: omega_weights(theta, theta_i, lambda_penalty=lambda_penalty))  

## `LocalBasis`

In [ ]:
max_mut = mut.max(axis = 0).values
min_mut = mut.min(axis = 0).values

def scaling(theta):
    return theta * (max_mut-min_mut) + min_mut

POD_var = LocalBasis(q=p+1,
                     module=w_POD, 
                     p_prime_index_list=np.arange(p+1), 
                     scaling=scaling)

DOD_var = LocalBasis(q=p+1,
                     module=dod, 
                     p_prime_index_list=[0,2], # the bad parameters are the same the DOD has been trained for 
                     scaling=scaling)

## Check compatibility

We can look at the following:

* the size of an output space;
* what happens when checking the K score.

### Size of the space

Define a random $\hat{\theta}$ in $[0,1]^q$.

In [ ]:
torch.manual_seed(seed)
theta_hat = torch.rand(pbad+pgood)

The expected size for both is a tensor of size $n \times N_A$.

In [ ]:
print("Size of the output space for POD_var:\t", POD_var(theta_hat).size())
print("Size of the output space for DOD_var:\t", DOD_var(theta_hat).size())

### K score

When computing the K score, we are measuring how adaptive the `module` is against all the parameters.
If the direction we compute the score is one of the directions the `DOD` has been trained for, both object will output a value different from 0.

In [ ]:
POD_K = POD_var.K_j_sup(0, theta_hat)
print(f"The value of the K score for 1/ρ computed by POD_var:\t{POD_K:.6e}")
DOD_K = DOD_var.K_j_sup(0, theta_hat)
print(f"The value of the K score for 1/ρ computed by DOD_var:\t{DOD_K:.6e}")

If instead the corresponding parameter is not one the `DOD` has not been trained for (in this case $\eta$), its score will be automatically assigned to 0.

In [ ]:
POD_K = POD_var.K_j_sup(1, theta_hat)
print(f"The value of the K score for η computed by POD_var:\t{POD_K:.6e}")
DOD_K = DOD_var.K_j_sup(1, theta_hat)
print(f"The value of the K score for η computed by DOD_var:\t{DOD_K:.6e}")